In [68]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from __future__ import absolute_import, division, print_function, unicode_literals
from IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc


In [69]:
# Load data

dftrain = pd.read_csv('data_train.csv')
dfeval = pd.read_csv('data_test.csv')
y_train = dftrain.pop('POP-4')
y_eval = dfeval.pop('POP-4')

CATEGORICAL_COLUMNS = ['project'] #'STA-7'
NUMERIC_COLUMNS = ['COM-1', 'COM-3', 'POP-4', 'STA-1', 'STA-2', 'STA-4', 'STA-5', 'STA-6','STA-10','STA-11','STA-12', 'TEC-1', 'TEC-2.1',
                    'TEC-2.2','TEC-2.3', 'TEC-2.4', 'TEC-4', 'TEC-5', 'SWQ-1', 'SWQ-4.1', 'SWQ-4.2', 'SWQ-4.3', 'SWQ-4.4', 'SWQ-4.5', 'SWQ-4.6', 'SWQ-4.7', 'init', 'end', 'frequency(weeks)', 'project_url']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
    vocabulary = dftrain[feature_name].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))


In [70]:
# Make input function
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds  # Move this line outside the if statement

    return input_function  # Move this line outside the input_function() function

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)


In [71]:
# Creating the model
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result['accuracy'])


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\klock\\AppData\\Local\\Temp\\tmpcbtl4s9q', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


TypeError: Expected binary or unicode string, got nan